In [76]:
!pip install simpy plotly

### Goal
- randomization of tasks duration
- structure of user scenario


In [77]:
import simpy
import datetime
import pandas as pd
import plotly.express as px
import logging
from enum import Enum

# new
import random
from itertools import repeat


In [78]:
class Metric(Enum):
    RW = "Requests Waiting"
    BS = "Busy Slots"
    AU = "Active Users"

In [79]:
log_filename = "logs-8.log"
mainLogger = logging.getLogger()
mainLogger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_filename, mode='w')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
mainLogger.addHandler(fhandler)
mainLogger.setLevel(logging.DEBUG)
mainLogger.debug("test")

In [80]:
class Scenario1:
    def __init__(self):
        self.name = "Sc1"
        self.tasks = []
        self.tasks.append( 
            { 'Name': "Process-1",
              'Duration': 5,
              'Res': 1
            })        
        self.tasks.append( 
            { 'Name': "Verify-1",
              'Duration': 2
            })

class Scenario2:
    def __init__(self):
        self.name = "Sc2"
        self.tasks = []
        self.tasks.append( 
            { 'Name': "Process-2.1",
              'Duration': 2,
              'Res': 1
            })        
        self.tasks.append( 
            { 'Name': "Verify-2.1",
              'Duration': 1
            })
        self.tasks.append( 
            { 'Name': "Wait-2.1",
              'Duration': 1
            })
        self.tasks.append( 
            { 'Name': "Process-2.2",
              'Duration': 3,
              'Res': 1
            })        
        self.tasks.append( 
            { 'Name': "Verify-2.2",
              'Duration': 1
            })


In [81]:
class User:
    #def __init__(self, id, env, res, monitor):
    def __init__(self, id, scenario, world):
        self.id = id
        self.scenario = scenario
        self._world = world
        self.taskid = 1
        self.create()
        # Start the run process everytime an instance is created.
        # create itself as a processs
        self.action = self._world.env.process(self.run())

    def create(self):
        self.enteringAt = self._world.env.now
        #self.name = f"User-{self.id}" 
        self.name = "User-%03d" % self.id
        mainLogger.info(f"creating user {self.name}")
        #self._world.monitor.report_new_user(self)
        self._world.user_monitor.report_new_user(self)

    def run_old(self):
        while True:
            taskname = "process_task"
            self.taskid += 1
            self.monitor.report_start(self.name, taskname, self.taskid)
            mainLogger.debug(f"{self.name} starts task at %d" % self.env.now)
            task_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.process_task(task_duration))
            mainLogger.debug(f"{self.name} ends task at %d" % self.env.now)
            self.monitor.report_stop(self.name, taskname, self.taskid)
     
            # The charge process has finished and
            # we can start driving again.
            mainLogger.debug(f"{self.name} starts verification at %d" % self.env.now)
            verification_duration = 2
            self.last_action_started_at = self.env.now
            yield self.env.timeout(verification_duration)
            mainLogger.debug(f"{self.name} ends verification at %d" % self.env.now)


    def run(self):
        while True:
            self.taskid += 1
            for task in self.scenario.tasks:
                taskname = task['Name']
                task_duration = task['Duration']
                mark = self._world.env.now
                mainLogger.debug(f"{self.name} starts task {taskname} at %d" % mark)

                if 'Res' in task:
                    self._world.user_monitor.report_start(
                            self.name, 
                            self.scenario, #new
                            taskname, 
                            self.taskid)
                    # We yield the process that process() returns
                    # to wait for it to finish
                    yield self._world.env.process(self.process_task(task_duration))
                    self._world.user_monitor.report_stop(
                            self.name, 
                            self.scenario, #new
                            taskname, 
                            self.taskid)
                else:
                    # wait some time even if no tracked
                    yield self._world.env.timeout(task_duration)

                mainLogger.debug(f"{self.name} ends task {taskname} at %d" % mark)
                 
    def process_task(self, duration):
        with self._world.res.request() as req:
            yield req
            yield self._world.env.timeout(duration)


In [82]:
class Clock:
    def __init__(self):
        self.base_epoch = datetime.datetime.now().timestamp()
        mainLogger.info(f"Clock created - base {self.base_epoch}")

    def to_date(self, tick):
        epoch_time = self.base_epoch + tick*60 #mn
        datetime_time = datetime.datetime.fromtimestamp(epoch_time)
        return datetime_time

In [83]:
class UsersMonitor:
    def __init__(self, world):
        # init parameters are self reported
        # start and stop events
        self.start_data = []
        self.stop_data = []
        # list of users
        self.users = []
        
    def report_new_user(self, user):
        self.users.append(user)            
        
    #def report_start(self, username, taskname, taskid):
    def report_start(self, username, scenario, taskname, taskid):
        mark = world.env.now
        self.start_data.append(
            dict(  
                StartMark=mark,
                Start=world.clock.to_date(mark),
                Username=username,
                Scenario=scenario.name, # new
                Task=taskname,
                TaskId=taskid
            )
        )       
        
    #def report_stop(self, username, taskname, taskid):
    def report_stop(self, username, scenario, taskname, taskid):
        mark = world.env.now
        self.stop_data.append(
            dict(  
                FinishMark=mark,
                Finish=world.clock.to_date(mark),
                Username=username,
                Scenario=scenario.name, # new
                Task=taskname,
                TaskId=taskid
            )
        )
        
                
    def collect(self):
        df_start = pd.DataFrame(self.start_data)
        df_stop = pd.DataFrame(self.stop_data)
        df = pd.merge(df_start, df_stop, how='left', 
                      #on = ['Username', 'Task', 'TaskId'])
                      on = ['Username', 'Scenario', 'Task', 'TaskId'])
        df['Duration'] = df['FinishMark'] - df['StartMark']
        return df


In [84]:
# wake up every tick and collect
class UsersGenerator:
    def __init__(self, world, max_nb_users=10):
        self._world = world
        self._max_nb_users = max_nb_users
        mainLogger.info("creating  user generator")
        self.data = []
        self.active_users = []
        self.user_count = 0
        # this will be used as a process
        self.action = world.env.process(self.run())

    def run(self):
        while True:
            if self.user_count < self._max_nb_users:
                self.create_user()
        
            self.report()

            tick_duration = 1
            yield self._world.env.timeout(tick_duration)

    def create_user(self):
        self.user_count += 1

        # new - npw can vary scenarios
        scenario = random.choice(self._world.scenarios)

        user = User(self.user_count, 
                    scenario,
                    #self._world.env, 
                    #self._world.res, 
                    #self._world.user_monitor).action
                    self._world)
        self.active_users.append(user)
        mark = self._world.env.now
        mainLogger.debug(f"{len(self.active_users)} active users at %d" % mark)
        
    def report(self):
        mark = self._world.env.now
        active_users_count = len(self.active_users)

        self.data.append(
            dict(  
                Mark=mark,
                Timestamp=self._world.clock.to_date(mark),
                Metric=Metric.AU.value,
                Value=active_users_count
            )
        )       
        
    
    def collect(self):
        return pd.DataFrame(self.data)


In [85]:
class SystemResource(simpy.Resource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        #print(f"create resource with capacity {self.capacity}")
        mainLogger.info(f"create resource with capacity {self.capacity}")

    def request(self, *args, **kwargs):
        #print("request resource at %d" % self._env.now)
        mainLogger.debug("request resource at %d" % self._env.now)
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        #print("release resource at %d" % self._env.now)
        mainLogger.debug("release resource at %d" % self._env.now)
        return super().release(*args, **kwargs)


In [86]:
# wake up every tick and collect
class SystemMonitoringAgent:
    def __init__(self, world):
        self._world = world
        mainLogger.info("creating agent")
        self.data = []
        # this will be used as a process
        self.action = world.env.process(self.run())

    def run(self):
        while True:

            mark = self._world.env.now
            occupied_slots = self._world.res.count
            requests_waiting = len(self._world.res.queue)
            
            mainLogger.debug(f"{occupied_slots} occupied slots at %d" % mark)
            mainLogger.debug(f"{requests_waiting} requests waiting at %d" % mark)

            self.data.append(
                dict(  
                    Mark=mark,
                    Timestamp=self._world.clock.to_date(mark),
                    Metric=Metric.BS.value,
                    Value=occupied_slots
                )
            )       
            self.data.append(
                dict(  
                    Mark=mark,
                    Timestamp=self._world.clock.to_date(mark),
                    Metric=Metric.RW.value,
                    Value=requests_waiting
                )
            ) 
            
            tick_duration = 1
            yield self._world.env.timeout(tick_duration)

        
                
    def collect(self):
        return pd.DataFrame(self.data)


In [87]:
class World:
    
    def __init__(self, scenarios, nb_users=20, resource_capacity=5):
        self.scenarios = scenarios
        self.env = simpy.Environment()
        self.clock = Clock()
        self.res = SystemResource(self.env, capacity=resource_capacity)
        self.user_monitor = UsersMonitor(self)
        self.user_gen = UsersGenerator(self, max_nb_users=nb_users)
        self.res_agent = SystemMonitoringAgent(self)

    def start(self, sim_duration = 20):
        self.env.run(until=sim_duration)

## Configuration

In [88]:
mainLogger.setLevel(logging.INFO)
#mainLogger.setLevel(logging.DEBUG)
        
scenarios = []
scenarios.extend(repeat(Scenario1(), 3))
scenarios.extend(repeat(Scenario2(), 2))

world = World(scenarios, # new
              nb_users = 30,
              resource_capacity = 5 )

In [89]:
world.start(sim_duration = 100)

# Time Series

In [90]:
df_system = world.res_agent.collect()
display(df_system)

,Mark,Metric,Timestamp,Value
0,0,Busy Slots,2021-09-12 12:15:04.492762,0
1,0,Requests Waiting,2021-09-12 12:15:04.492762,0
2,1,Busy Slots,2021-09-12 12:16:04.492762,2
3,1,Requests Waiting,2021-09-12 12:16:04.492762,0
4,2,Busy Slots,2021-09-12 12:17:04.492762,2
5,2,Requests Waiting,2021-09-12 12:17:04.492762,0
6,3,Busy Slots,2021-09-12 12:18:04.492762,3
7,3,Requests Waiting,2021-09-12 12:18:04.492762,0
8,4,Busy Slots,2021-09-12 12:19:04.492762,3
9,4,Requests Waiting,2021-09-12 12:19:04.492762,0


## Occupied slots

In [91]:
#df = dfs['occupied_slots']
df = df_system[ df_system.Metric == Metric.BS.value ]
fig = px.bar(df, x='Timestamp', y='Value')
fig.show()

## Requests waiting

In [92]:
#df = dfs['requests_waiting']
df = df_system[ df_system.Metric == Metric.RW.value ]
fig = px.bar(df, x='Timestamp', y='Value')
fig.show()

## Active Users

In [93]:
df_users_active = world.user_gen.collect()
display(df_users_active)

,Mark,Metric,Timestamp,Value
0,0,Active Users,2021-09-12 12:15:04.492762,1
1,1,Active Users,2021-09-12 12:16:04.492762,2
2,2,Active Users,2021-09-12 12:17:04.492762,3
3,3,Active Users,2021-09-12 12:18:04.492762,4
4,4,Active Users,2021-09-12 12:19:04.492762,5
5,5,Active Users,2021-09-12 12:20:04.492762,6
6,6,Active Users,2021-09-12 12:21:04.492762,7
7,7,Active Users,2021-09-12 12:22:04.492762,8
8,8,Active Users,2021-09-12 12:23:04.492762,9
9,9,Active Users,2021-09-12 12:24:04.492762,10


In [94]:
fig = px.bar(df_users_active, x='Timestamp', y='Value')
fig.show()

# Users activity

## Users chronogram

In [95]:
df_users = world.user_monitor.collect()
display(df_users)

,Scenario,Start,StartMark,Task,TaskId,Username,Finish,FinishMark,Duration
0,Sc2,2021-09-12 12:15:04.492762,0,Process-2.1,2,User-001,2021-09-12 12:17:04.492762,2.0,2.0
1,Sc1,2021-09-12 12:16:04.492762,1,Process-1,2,User-002,2021-09-12 12:21:04.492762,6.0,5.0
2,Sc2,2021-09-12 12:17:04.492762,2,Process-2.1,2,User-003,2021-09-12 12:19:04.492762,4.0,2.0
3,Sc1,2021-09-12 12:18:04.492762,3,Process-1,2,User-004,2021-09-12 12:23:04.492762,8.0,5.0
4,Sc1,2021-09-12 12:19:04.492762,4,Process-1,2,User-005,2021-09-12 12:24:04.492762,9.0,5.0
5,Sc2,2021-09-12 12:19:04.492762,4,Process-2.2,2,User-001,2021-09-12 12:22:04.492762,7.0,3.0
6,Sc1,2021-09-12 12:20:04.492762,5,Process-1,2,User-006,2021-09-12 12:25:04.492762,10.0,5.0
7,Sc1,2021-09-12 12:21:04.492762,6,Process-1,2,User-007,2021-09-12 12:26:04.492762,11.0,5.0
8,Sc2,2021-09-12 12:21:04.492762,6,Process-2.2,2,User-003,2021-09-12 12:25:04.492762,10.0,4.0
9,Sc1,2021-09-12 12:22:04.492762,7,Process-1,2,User-008,2021-09-12 12:28:04.492762,13.0,6.0


In [96]:
#new
#fig = px.timeline(df_users, x_start="Start", x_end="Finish", y="Username")
fig = px.timeline(df_users, x_start="Start", x_end="Finish", y="Username", color="Scenario")
#fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.update_yaxes(categoryorder='category descending')
fig.show()

## Users activity distribution

In [97]:
df_users[["Duration"]].describe()

,Duration
count,112.000000
mean,17.830357
std,7.547911
min,2.000000
25%,12.000000
50%,22.000000
75%,24.000000
max,26.000000


In [98]:
mainLogger.debug(f"max={max}")
fig = px.histogram(df_users, x="Duration", nbins=20)
fig.show()

In [99]:
# when starts waiting - more or less 8 users
# as much as processes - more or less 13 users

# Logs

In [100]:
with open(log_filename) as log:
            print(log.read())

2021-09-12 12:15:04,405 - root - DEBUG - test
2021-09-12 12:15:04,492 - root - INFO - Clock created - base 1631441704.492762
2021-09-12 12:15:04,493 - root - INFO - create resource with capacity 5
2021-09-12 12:15:04,493 - root - INFO - creating  user generator
2021-09-12 12:15:04,493 - root - INFO - creating agent
2021-09-12 12:15:04,497 - root - INFO - creating user User-001
2021-09-12 12:15:04,498 - root - INFO - creating user User-002
2021-09-12 12:15:04,498 - root - INFO - creating user User-003
2021-09-12 12:15:04,499 - root - INFO - creating user User-004
2021-09-12 12:15:04,499 - root - INFO - creating user User-005
2021-09-12 12:15:04,499 - root - INFO - creating user User-006
2021-09-12 12:15:04,499 - root - INFO - creating user User-007
2021-09-12 12:15:04,500 - root - INFO - creating user User-008
2021-09-12 12:15:04,500 - root - INFO - creating user User-009
2021-09-12 12:15:04,500 - root - INFO - creating user User-010
2021-09-12 12:15:04,500 - root - INFO - creating user